<a href="https://colab.research.google.com/github/SorahP/ddPn08_RVC-WebUI-colab/blob/main/ddPn08_RVC_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/liujing04/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI.ipynb)

In [ ]:
#@title GPU確認
!nvidia-smi

In [ ]:
#@title ライブラリをインストール
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-gpu fairseq gradio ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2

In [ ]:
#@title rvc-webuiをダウンロード

!git clone https://github.com/ddPn08/rvc-webui.git
%cd /content/rvc-webui/
!mkdir -p pretrained uvr5_weights
!git pull

In [ ]:
#@title aria2をインストール ＆ 必要なファイルをダウンロード
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/rvc-webui/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/rvc-webui/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/rvc-webui/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/rvc-webui/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/rvc-webui/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/rvc-webui/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/rvc-webui/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/rvc-webui/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/rvc-webui/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/rvc-webui/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/rvc-webui/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/rvc-webui/pretrained -o f0G48k.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/rvc-webui/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/rvc-webui/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

In [ ]:
#@title hubert_baseをダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/rvc-webui -o hubert_base.pt

In [ ]:
#@title ドライブをマウント

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title データセットを展開

#@markdown データセットのフルパス
DATASET = "/content/drive/MyDrive/RVC-1/Dataset1.zip"  #@param {type:"string"}

!mkdir -p /content/dataset
!unzip -d /content/dataset -B {DATASET}

In [ ]:
#@title ファイルのリネーム
!ls -a /content/dataset/
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/dataset/*.*~*

In [ ]:
#@title 重みをドライブからダウンロード
#@markdown Discriminatorのパス
discriminator = "/content/drive/MyDrive/RVC-1/f0G48k768_jphubert_tuned.pth"  #@param {type:"string"}
#@markdown Generatorのパス
generator = "/content/drive/MyDrive/RVC-1/f0D48k768_jphubert_tuned.pth"  #@param {type:"string"}
!cp {discriminator} /content/rvc-webui/pretrained/
!cp {generator} /content/rvc-webui/pretrained/f0G48k768_jphubert_tuned.pth

In [ ]:
#@title WebUIを起動
%cd /content/rvc-webui
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
!python3 launch.py --share --colab --pycmd python3

In [ ]:
#@title 手动将训练后的模型文件备份到谷歌云盘
#@markdown 需要自己查看logs文件夹下模型的文件名，手动修改下方命令末尾的文件名

#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 模型epoch
MODELEPOCH = 9600  #@param {type:"integer"}

!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/drive/MyDrive/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/drive/MyDrive/{MODELNAME}_G_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index /content/drive/MyDrive/
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy /content/drive/MyDrive/

!cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth /content/drive/MyDrive/{MODELNAME}{MODELEPOCH}.pth

In [ ]:
#@title 从谷歌云盘恢复pth
#@markdown 需要自己查看logs文件夹下模型的文件名，手动修改下方命令末尾的文件名

#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 模型epoch
MODELEPOCH = 7500  #@param {type:"integer"}

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

!cp /content/drive/MyDrive/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!cp /content/drive/MyDrive/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth
!cp /content/drive/MyDrive/*.index /content/
!cp /content/drive/MyDrive/*.npy /content/
!cp /content/drive/MyDrive/{MODELNAME}{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth

In [ ]:
#@title 手动预处理（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 采样率
BITRATE = 48000  #@param {type:"integer"}
#@markdown 使用的进程数
THREADCOUNT = 8  #@param {type:"integer"}

!python3 trainset_preprocess_pipeline_print.py /content/dataset {BITRATE} {THREADCOUNT} logs/{MODELNAME} True


In [ ]:
#@title 手动提取特征（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 使用的进程数
THREADCOUNT = 8  #@param {type:"integer"}
#@markdown 音高提取算法
ALGO = "harvest"  #@param {type:"string"}

!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}

!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME}


In [ ]:
#@title 手动训练（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 使用的GPU
USEGPU = "0"  #@param {type:"string"}
#@markdown 批大小
BATCHSIZE = 32  #@param {type:"integer"}
#@markdown 停止的epoch
MODELEPOCH = 3200  #@param {type:"integer"}
#@markdown 保存epoch间隔
EPOCHSAVE = 100  #@param {type:"integer"}
#@markdown 采样率
MODELSAMPLE = "48k"  #@param {type:"string"}
#@markdown 是否缓存训练集
CACHEDATA = 1  #@param {type:"integer"}
#@markdown 是否仅保存最新的ckpt文件
ONLYLATEST = 0  #@param {type:"integer"}

!python3 train_nsf_sim_cache_sid_load_pretrain.py -e lulu -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g {USEGPU} -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained/f0G{MODELSAMPLE}.pth -pd pretrained/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA}


In [ ]:
#@title 删除其它pth，只留选中的（慎点，仔细看代码）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 选中模型epoch
MODELEPOCH = 9600  #@param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*.pth

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth 
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

In [ ]:
#@title 清除项目下所有文件，只留选中的模型（慎点，仔细看代码）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 选中模型epoch
MODELEPOCH = 9600  #@param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm -rf /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth 
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}